In [ ]:
# Fix pybel import path
try:
    import sys
    sys.modules['pybel'] = __import__('openbabel', globals(), locals(), ['pybel']).pybel
except Exception:
    pass

In [ ]:
import ipywidgets as ipw
from aiida.orm import StructureData
from aiida.orm import load_node
from aiidalab_widgets_base import CodQueryWidget, SmilesWidget, StructureExamplesWidget
from aiidalab_widgets_base import OptimadeQueryWidget
from aiidalab_widgets_base import StructureBrowserWidget, StructureManagerWidget, StructureUploadWidget
from aiidalab_widgets_base import WizardAppWidget

from aiidalab_qe.process import WorkChainSelector
from aiidalab_qe.steps import SubmitQeAppWorkChainStep
from aiidalab_qe.steps import ViewQeAppWorkChainStatusAndResultsStep
from aiidalab_qe.structures import StructureSelectionStep


OptimadeQueryWidget.title = "OPTIMADE"  # monkeypatch


# Create the application steps
structure_manager_widget = StructureManagerWidget(
    importers=[
        StructureUploadWidget(title="From computer"),
        CodQueryWidget(title="COD"),
        StructureBrowserWidget(title="AiiDA database"),
        OptimadeQueryWidget(embedded=True),
        SmilesWidget(title="SMILES"),  # requires OpenBabel! 
        StructureExamplesWidget(
            title="From Examples",
            examples=[
                ("Silicon", "miscellaneous/structures/Si.xyz"),
                ("Silicon oxide", "miscellaneous/structures/SiO2.xyz"),
            ]),
    ],
    node_class='StructureData',
)
structure_selection_step = StructureSelectionStep(manager=structure_manager_widget)
structure_selection_step.auto_advance = True
submit_qe_app_work_chain_step = SubmitQeAppWorkChainStep()
submit_qe_app_work_chain_step.auto_advance = True
view_qe_app_work_chain_status_and_results_step = ViewQeAppWorkChainStatusAndResultsStep()

# Link the application steps
ipw.dlink((structure_selection_step, 'confirmed_structure'), (submit_qe_app_work_chain_step, 'input_structure'))
ipw.dlink((submit_qe_app_work_chain_step, 'process'), (view_qe_app_work_chain_status_and_results_step, 'process'))

# Add the application steps to the application
app = WizardAppWidget(
    steps=[
        ('Select structure', structure_selection_step),
        ('Submit work chain', submit_qe_app_work_chain_step),
        ('Status & Results', view_qe_app_work_chain_status_and_results_step),
    ])

# Reset all subsequent steps in case that a new structure is selected
def _observe_structure_selection(change):
    with structure_selection_step.hold_sync():
        if structure_selection_step.confirmed_structure is not None and \
                structure_selection_step.confirmed_structure != change['new']:
            app.reset()
structure_selection_step.observe(_observe_structure_selection, 'structure')

# Add process selection header
work_chain_selector = WorkChainSelector(layout=ipw.Layout(width='auto'))

def _observe_process_selection(change):
    if change['old'] == change['new']:
        return
    pk = change['new']
    if pk is None:
        app.reset()
        app.selected_index = 0
    else:
        process = load_node(pk)
        with structure_manager_widget.hold_sync():
            with structure_selection_step.hold_sync():
                structure_manager_widget.structure = process.inputs.structure
                structure_selection_step.structure = process.inputs.structure
                structure_selection_step.confirmed_structure = process.inputs.structure
                submit_qe_app_work_chain_step.process = process
        
work_chain_selector.observe(_observe_process_selection, 'value')    
ipw.dlink((submit_qe_app_work_chain_step, 'process'), (work_chain_selector, 'value'),
          transform=lambda node: None if node is None else node.pk)

app_with_work_chain_selector = ipw.VBox(children=[work_chain_selector, app])

# Add banner in case that pseudo potentials must be installed
sssp_warning_banner = ipw.HTML("""
    <div class="alert alert-warning alert-dismissible">
    <a href="#" class="close" data-dismiss="alert" aria-label="close">&times;</a>
    <p><strong>WARNING</strong>
    The pseudo potential families required for this app are not yet installed.</p>
    <p>In step 2: click on 'Pseudopotential', and then on 'Install pseudos' to install them.</p>
    </div>""" if not submit_qe_app_work_chain_step.pseudo_family_selector.installed else "")

display(sssp_warning_banner, app_with_work_chain_selector)